In [ ]:

def ransomhouse(json_data, days):
    """Filtra as entradas com actionDate dentro dos últimos 'days' dias."""

    # Definindo a data limite
    hoje = datetime.now()
    dias_atras = hoje - timedelta(days=days)

    filtered_data = []
    
    for entry in json_data.get('data', []):
        # Verifica se a chave 'actionDate' existe
        if 'actionDate' in entry:
            date_str = entry['actionDate']
            
            # Verificar se a string não está vazia
            if date_str:
                # Converter a string da data para um objeto datetime
                try:
                    date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                    # Verificar se a data está dentro do intervalo
                    if dias_atras <= date_obj <= hoje:
                        filtered_data.append({
                            'header': entry.get('header', ''),
                            'url': entry.get('url', ''),
                            'actionDate': date_str
                        })
                except ValueError:
                    # Ignorar se a data não for válida
                    continue

    return filtered_data


In [ ]:

def monti(html_content, dias):
    # Criando um objeto BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Inicializando a lista de resultados
    results = []

    # Definindo o limite de datas (D-7)
    hoje = datetime.now()
    limite = hoje - timedelta(days=dias)

    # Encontrando todas as divs com a classe 'col-auto published'
    divs = soup.find_all('div', class_='col-auto published')
    
    # Iterando sobre as divs e verificando se a data é válida
    for div in divs:
        try:
            # Convertendo o texto da div para uma data
            data_texto = div.text.strip()
            
            # Ajuste o formato para incluir horas, minutos e segundos
            data_publicacao = datetime.strptime(data_texto, '%Y-%m-%d %H:%M:%S')
            
            # Verificando se a data está dentro do intervalo (últimos D-7 dias)
            if limite <= data_publicacao <= hoje:
                # Encontrando o título anterior à div
                h5 = div.find_previous('h5')
                title = h5.text.strip() if h5 else 'Título não encontrado'
                
                # Adicionando o título e a data à lista de resultados
                results.append({'title': title, 'date': data_texto})
        except ValueError as e:
            # Ignorar se o valor não for uma data válida, mas logar para depuração
            print(f"Erro ao converter data: {data_texto}. Erro: {e}")
            continue

    return results

In [ ]:
def play(html_content, days):
    # Criando um objeto BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Data atual e data de corte para 7 dias atrás
    current_date = datetime.now()
    cutoff_date = current_date - timedelta(days)

    # Coletando os dados
    data_rows = []

    for th in soup.find_all('th', class_='News'):
        # Título é o texto do th, excluindo as tags filhas
        title = th.contents[0].strip()  # Apenas o texto do <th> antes do <div>

        # Encontrando o link
        link_div = th.find('i', class_='link')
        link = link_div.next_sibling.strip() if link_div and link_div.next_sibling else None
        
        # Encontrando a data de publicação
        publication_date_div = th.find(string=lambda text: text and 'publication date:' in text)
        publication_date = publication_date_div.strip().split(':')[-1].strip() if publication_date_div else None
        
        # Verificando se a data de publicação está nos últimos 7 dias
        if publication_date:
            pub_date = datetime.strptime(publication_date, '%Y-%m-%d')
            if pub_date >= cutoff_date:  # Apenas incluir se a data de publicação for maior ou igual à data de corte
                data_rows.append({'title': title, 'link': link, 'publication_date': publication_date})

    return data_rows

In [ ]:
def handala(html_content, dias):
    # Fazendo o parsing do HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Definindo o limite de datas (D-7)
    hoje = datetime.now()
    limite = hoje - timedelta(days=dias)

    # Coletando informações
    results = []
    for li in soup.find_all('li', class_='wp-block-post'):
        title = li.find('h2').find('a').text
        time_tag = li.find('time')
        
        if time_tag:
            date = time_tag.get('datetime').split('T')[0]  # Pegando apenas a data
            date_obj = datetime.strptime(date, '%Y-%m-%d')

            # Verificando se a data está dentro do intervalo dos últimos D-7 dias
            if limite <= date_obj <= hoje:
                description = li.find('p', class_='wp-block-post-excerpt__excerpt').text
                
                # Extraindo o site da descrição
                site_match = re.search(r'\b(?:https?://|www\.)?([\w.-]+(?:\.[a-z]{2,}))\b', description)
                site = site_match.group(0) if site_match else None
                
                # Adicionando os resultados
                results.append({
                    'title': title,
                    'date': date,
                    'site': site
                })

    return results

In [ ]:

def blackbyte(html, days):
    # Parsear o HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    # Obter o nome da empresa (caption)
    nome_empresa = soup.find('caption', class_='target-name').get_text()

    # Obter todas as tags <td> que contêm datas
    datas_td = soup.find_all('td')
    
    # Lista para armazenar os dicionários de resultados
    resultados = []
    
    # Definir o intervalo de datas (últimos 7 dias)
    hoje = datetime.now()
    limite = hoje - timedelta(days)
    
    # Iterar sobre as datas e verificar se estão dentro do intervalo
    for td in datas_td:
        try:
            # Tentar converter o texto da tag <td> para datetime
            data = datetime.strptime(td.get_text(), '%Y-%m-%d %H:%M')
            
            # Se a data for nos últimos 7 dias, adicionar ao resultado
            if limite <= data <= hoje:
                resultados.append({
                    "empresa": nome_empresa,
                    "data": td.get_text()
                })
        except ValueError:
            # Ignorar se o valor não for uma data válida
            continue

    # Retornar a lista de dicionários
    return resultados
